<a href="https://www.kaggle.com/code/lukalafaye/hub-eau-dataset?scriptVersionId=211026657" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import requests
import pandas as pd

def fetch_all_data(url, output_file, params, date_key_api, date_key):
    size = 20000
    response = requests.get(url, params)
    done = True
    if response.status_code not in  [200, 206]:
        done = False
        print(f"Error: {response.status_code} - {response.text}")
        
    with open(output_file, 'wb') as f : 
        f.write(response.content)

    stop = 20000
    while True:

        if stop != size : 
            done = True
            break

        with open('temp.csv', 'wb') as f : 
            f.write(response.content)

        df = pd.read_csv('temp.csv', sep=";")
        
        stop = len(df)
        
        # the dataset is already sorted by date
        
        last_date = df.tail(1)[date_key].iloc[0]
        # Convertir en datetime
        last_date = pd.to_datetime(last_date)
        print("last date:", last_date)
        # Ajouter un jour
        next_date = last_date + pd.Timedelta(days=1)
        print("next:", next_date)


        print(last_date)
        params[date_key_api] = next_date
        response = requests.get(url, params)

        if response.status_code not in  [200, 206]:
            done = False
            print(f"Error: {response.status_code} - {response.text}")
            break

        with open(output_file, 'ab') as f : 
            f.write(response.content)
        
    print("DONE", done)
    return 

In [6]:
params = {
    "size" : 20000,
    "code_departement": "75,77,78,91,92,93,94,95",
    "fields": "code_station,libelle_station,code_commune,libelle_commune,latitude,longitude,code_ecoulement,libelle_ecoulement,date_observation",
    "date_observation_min": "2023-01-01",
}

date_key_api = "date_observation_min"
date_key = "date_observation"

url = "https://hubeau.eaufrance.fr/api/v1/ecoulement/observations.csv"
output_file = "analyses_ec.csv"

fetch_all_data(url, output_file, params, date_key_api, date_key)

last date: 2023-03-22 00:00:00
next: 2023-03-23 00:00:00
2023-03-22 00:00:00
DONE True


In [7]:
params = {
    "size" : 20000,
    "fields": (
        "code_commune,nom_commune,date_prelevement,libelle_parametre,"
        "resultat_numerique,libelle_unite,conclusion_conformite_prelevement"
    ),
    "code_departement": "75,77,78,91,92,93,94,95",
    "date_min_prelevement": "2014-01-01",
}

date_key_api = "date_min_prelevement"
date_key = "date_prelevement"

url = "https://hubeau.eaufrance.fr/api/v1/qualite_eau_potable/resultats_dis.csv"


output_file = "analyses_qu.csv"

fetch_all_data(url, output_file, params, date_key_api, date_key)

last date: 2024-09-19 11:58:00+00:00
next: 2024-09-20 11:58:00+00:00
2024-09-19 11:58:00+00:00
last date: 2024-09-20 08:05:00+00:00
next: 2024-09-21 08:05:00+00:00
2024-09-20 08:05:00+00:00
DONE True


In [ ]:
import requests
import pandas as pd

# Juste un code réseau pour cette api...

url = "https://hubeau.eaufrance.fr/api/v1/ecoulement/observations"
params = {
    "format": "json",
    "code_departement": "75,77,78,91,92,93,94,95",
    "fields": "code_station,libelle_station,code_commune,libelle_commune,latitude,longitude,code_ecoulement,libelle_ecoulement,date_observation",
    "date_observation_min": "2014-01-01",
    "date_observation_max": "2024-12-31"
}
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()["data"]
    
    df_ecoulement = pd.DataFrame(data)
    
    """
    df_ecoulement["code_reseau"] = df_ecoulement["code_reseau"].apply(
        lambda x: x if isinstance(x, list) else [x]
    )
    df_ecoulement_expanded = df_ecoulement.explode("code_reseau")
    """ 
    
    df_ecoulement_expanded=df_ecoulement
    df_ecoulement_expanded.to_csv("ile_de_france_ecoulement_2022-2024.csv", index=False)
    print("Data saved to ile_de_france_ecoulement_2022-2024.csv")
else:
    print(f"Failed to fetch data: {response.status_code} - {response.text}")

df_ecoulement_expanded["code_commune"].unique()

In [18]:
import requests
import pandas as pd

url = "https://hubeau.eaufrance.fr/api/v1/qualite_eau_potable/resultats_dis"

params = {
    "size" : 20000,
     "page" : 1,
    "fields": (
        "code_commune,nom_commune,date_prelevement,libelle_parametre,"
        "resultat_numerique,libelle_unite,conclusion_conformite_prelevement"
    ),
    "code_departement": "75,77,78,91,92,93,94,95",
    "date_min_prelevement": "2014-01-01",
    "date_max_prelevement": "2024-01-01"
}

all_data = []

while True:
    response = requests.get(url, params=params)
    if response.status_code in [200, 206]:
        response_data = response.json()
        data = response_data.get("data", [])
        all_data.extend(data)
        next_page = response_data.get("next")
        if not next_page:
            break
        params["page"] += 1
        break # only one page
    else:
        print(f"Failed to fetch data on page {params['page']}: {response.status_code} - {response.text}")
        break

df_qualite = pd.DataFrame(all_data)
df_qualite = df_qualite[df_qualite["libelle_unite"] != "SANS OBJET"]
df_qualite["conclusion_conformite_prelevement"] = df_qualite["conclusion_conformite_prelevement"].apply(
    lambda x: 1 if "Eau d'alimentation conforme" in x else 0
)

"""
df_qualite["code_reseau"] = df_qualite["code_reseau"].apply(
    lambda x: x if isinstance(x, list) else [x]
)
df_qualite_expanded = df_qualite.explode("code_reseau")
"""

df_qualite_expanded = df_qualite
df_qualite_expanded.to_csv("ile_de_france_water_quality_2022-2024.csv", index=False)
print("Data saved to ile_de_france_water_quality_2022-2024.csv")

Data saved to ile_de_france_water_quality_2022-2024.csv


In [41]:
import requests
import pandas as pd
from io import StringIO

# Base URL for the API
url = "https://hubeau.eaufrance.fr/api/v1/qualite_eau_potable/resultats_dis.csv"

# Parameters for the API request
params = {
    "size": 5000,  # Adjusted to a smaller size to comply with API limits
    "fields": (
        "code_commune,nom_commune,date_prelevement,libelle_parametre,"
        "resultat_numerique,libelle_unite,conclusion_conformite_prelevement"
    ),
    "code_departement": "75,77,78,91,92,93,94,95",
    "date_min_prelevement": "2020-01-01",
    "date_max_prelevement": "2024-01-01",
    "page": 1  # Start with the first page
}

# Headers for the request
headers = {"accept": "text/csv"}

# Initialize an empty DataFrame to store all data
all_data = pd.DataFrame()

while True:
    # Make the API call
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code in [200, 206]:  # Handle both 200 and 206 status codes
        # Debugging: Print first 500 characters of response to inspect formatting
        print(response.text[:500])
        
        try:
            # Parse the CSV response with error handling for malformed rows
            csv_data = StringIO(response.text)
            df_page = pd.read_csv(csv_data, on_bad_lines="skip")  # Skips bad lines
            
            # Append the page data to the main DataFrame
            all_data = pd.concat([all_data, df_page], ignore_index=True)
        except Exception as e:
            print(f"Error processing page {params['page']}: {e}")
            break
        
        # Check if fewer rows are returned than the requested size, indicating the last page
        if len(df_page) < params["size"]:
            break
        
        # Increment the page number for the next request
        params["page"] += 1
    else:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        break

# Process the data if available
if not all_data.empty:
   all_data.to_csv("data.csv")

"libelle_parametre";"resultat_numerique";"libelle_unite";"code_commune";"nom_commune";"date_prelevement";"conclusion_conformite_prelevement"
"Conductivité à 25°C";"552.0";"µS/cm";"94078";"VILLENEUVE-SAINT-GEORGES";"2023-12-29T15:55:00Z";"Eau d'alimentation conforme aux exigences de qualité en vigueur pour l'ensemble des paramètres mesurés."
"Température de l'eau";"11.6";"°C";"94078";"VILLENEUVE-SAINT-GEORGES";"2023-12-29T15:55:00Z";"Eau d'alimentation conforme aux exigences de qualité en vigueur


In [14]:
df_qualite["code_commune"].unique()

array(['94016', '94003', '91377', '94037', '91339', '91272', '94038',
       '91471', '94043', '94079', '91570', '93078', '93007', '91549',
       '94068', '77260', '94081', '91103', '94054', '93010', '77289',
       '77514', '77445', '94041', '93027', '95018', '93070', '77296',
       '91469', '91521', '95063', '93066', '75056', '77147', '77288',
       '77509', '93045', '77243', '91589', '91533', '77285', '77101',
       '77414', '93001', '93048', '77111', '77019', '77012', '77435',
       '95572', '94074', '91390', '77359', '94058', '94044', '93073',
       '93059', '77279', '77130', '93064', '77131', '77194', '77210',
       '77249', '93050', '77305', '93072', '77358', '77448', '93015',
       '95500', '77251', '77172', '93051', '92064', '77284', '77122',
       '94046', '94002', '91345', '77108', '78640', '77083', '91156',
       '78646', '78158', '78418', '95353', '91292', '78311', '95151',
       '95304', '77316', '91692', '78586', '95199', '77132', '93079',
       '78358', '783

In [4]:
a = df_qualite_expanded["code_commune"].unique()
b = df_ecoulement_expanded["code_commune"].unique()

def intersection(a, b):
    return [ e for e in a if e in b ]

print(intersection(b, a))

['78536', '77060', '91573', '77384', '77076', '77131', '77363']


In [5]:
df_qualite_expanded["code_commune"] = df_qualite_expanded["code_commune"].astype(str)
df_ecoulement_expanded["code_commune"] = df_ecoulement_expanded["code_commune"].astype(str)

df_merged = pd.merge(
    df_qualite_expanded,
    df_ecoulement_expanded,
    on="code_commune",
    how="inner",
    suffixes=("_quality", "_ecoulement")
)

df_merged.to_csv("ile_de_france_merged.csv", index=False)
print("Merged data saved to ile_de_france_merged.csv")

Merged data saved to ile_de_france_merged.csv


In [16]:
import requests
import pandas as pd

url_contaminants = "https://hubeau.eaufrance.fr/api/v1/surveillance_littoral/contaminants_chimiques"
params_contaminants = {
    "format": "json",
    "fields": (
        "code_lieusurv,libelle_lieusurv,longitude,latitude,"
        "date_prel,code_parametre,libelle_parametre,"
        "resultat_analyse,libelle_unite_resultat"
    ),
    "date_min_prel": "2014-01-01T00:00:00",
    "date_max_prel": "2024-12-31T23:59:59",
    "size": 500,
    "page": 1,
}

all_contaminants_data = []
current_page = 1

while True:
    response_contaminants = requests.get(url_contaminants, params=params_contaminants)
    if response_contaminants.status_code in [200, 206]:
        response_data = response_contaminants.json()
        data_contaminants = response_data.get("data", [])
        
        if not data_contaminants:
            break
        
        all_contaminants_data.extend(data_contaminants)
        
        next_page = response_data.get("next")
        if not next_page or current_page * params_contaminants["size"] >= 20000:
            break
        
        params_contaminants["page"] += 1
        current_page += 1
    else:
        break

if all_contaminants_data:
    df_contaminants = pd.DataFrame(all_contaminants_data)
    selected_columns = [
        "code_lieusurv", "libelle_lieusurv", "longitude", "latitude", 
        "date_prel", "code_parametre", 
        "libelle_parametre", "resultat_analyse", "libelle_unite_resultat"
    ]
    df_contaminants_filtered = df_contaminants[selected_columns]

    df_contaminants_expanded = df_contaminants_filtered
    df_contaminants_expanded.to_csv("littoral_contaminants_2022-2024.csv", index=False)
    print("Data saved to littoral_contaminants_2022-2024.csv")

Data saved to littoral_contaminants_2022-2024.csv
